# Data: Ad Sales Data
# Use Case: Revenue Prediction
# Model: Regression Models

Code link: https://www.kaggle.com/code/akshaysunil07/ad-tech-revenue-regression/notebook 

# Installing packages section

In [ ]:
# Installing packages set for without init script
!pip install --upgrade pip

!pip install "snowflake-connector-python[pandas]" "snowflake-snowpark-python[pandas]" snowflake-snowpark-python==1.9.0 fosforio fosforml numpy pandas matplotlib scikit-learn xgboost python-dateutil 
#!pip install tqdm 
!pip install --upgrade --q snowflake-snowpark-python==1.9.0
!pip uninstall urllib3 -y
!pip install urllib3==1.26.15
!pip install seaborn

# Restart and clear outputs

# Importing packages

In [2]:
from fosforio import snowflake
from fosforml import *
from fosforml.constants import MLModelFlavours
from fosforio import get_dataframe

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
pd.set_option('display.max_columns', 500)
import seaborn as sns

import warnings; warnings.simplefilter('ignore')
from joblib import dump, load
import requests
#from tqdm import tqdm
import time
import calendar
import configparser
from dateutil.relativedelta import relativedelta
import datetime

from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from time import sleep
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from dateutil.easter import easter
from scipy.optimize import minimize_scalar
from scipy.optimize import curve_fit
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
%matplotlib inline

Connection manager service url initialised to http://fdc-project-manager:80/project-manager
If you need to update its value then update the variable CONNECTION_MANAGER_BASE_URL in os env.


/opt/conda/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [4]:
pip freeze

aiohttp==3.9.3
aiosignal==1.3.1
alembic==1.13.1
anyio==4.2.0
archspec==0.2.1
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
asn1crypto==1.5.1
asttokens==2.4.1
async-generator==1.10
async-timeout==4.0.3
attrs==23.2.0
Babel==2.14.0
beautifulsoup4==4.12.3
bleach==4.1.0
boltons==23.0.0
bqplot==0.12.42
Brotli==1.0.9
certifi==2024.7.4
certipy==0.1.3
cffi==1.16.0
charset-normalizer==3.3.2
click==8.1.7
clickclick==20.10.2
cloudpickle==3.0.0
comm==0.2.1
conda==23.11.0
conda-content-trust==0.1.3
conda-libmamba-solver==23.12.0
conda-package-handling==2.2.0
conda-package-streaming==0.9.0
configparser==7.0.0
connexion==2.6.0
contourpy==1.2.1
cryptography==42.0.8
cycler==0.12.1
Cython==0.29.15
decorator==5.1.1
defusedxml==0.7.1
dill==0.3.8
distro==1.8.0
ds-lime==0.1.1.27
entrypoints==0.4
exceptiongroup==1.2.0
executing==2.0.1
fastjsonschema==2.19.1
filelock==3.15.4
flasgger==0.9.5
Flask==2.2.5
fonttools==4.53.1
fosforio==1.0.1
fosforml==1.0.1
frozenlist==1.4.1
greenlet==3.0.3
gunicorn==19.9.0
idna

In [ ]:
!pip install cryptography

# Importing data from snowflake

In [3]:
#from fosforio import snowflake
#from fosforio import get_dataframe
snowflake.get_connection(connection_name="ME_AD_SALES_CXN")
#df = get_dataframe("AD_SALES_IMP")
df_all = get_dataframe("AD_TECH_INPUT")

Connection details fetched: {'params': {'READER': {'connectionId': '87881a55-c8db-4d18-afa7-35fdcdf7ecff', 'connectionSources': {'sourceId': '210d73c1-4da7-487d-8a5c-05c8d2025269', 'connectionType': 'SNOWFLAKE', 'sourceName': 'RDBMS', 'enable': 1, 'displayName': 'SNOWFLAKE', 'displayOrder': 5.0}, 'createdBy': 'ms.akshaya@fosfor.com', 'createdOn': '2024-07-17T14:15:29.278+00:00', 'updatedOn': '2024-07-31T11:10:03.540+00:00', 'name': 'ME_AD_SALES_CXN', 'autosyncable': False, 'autopublishable': False, 'frequency': None, 'assetConfigurations': [], 'publishableNodes': [], 'autoPublishableNodes': None, 'isPrivate': False, 'isExpired': False, 'autoSyncFrequency': None, 'application': None, 'sourceDataRefreshFrequency': None, 'defaultDb': 'ME_DB', 'defaultSchema': 'ME_AD_SALES_SCHEMA', 'recommendedPrivileges': {'DECISION_MODULE': {'grant_on_warehouse_usage_to_role': True, 'grant_on_schema_to_role': True, 'create_stage_internal': True, 'create_udfs': True, 'grant_on_database_to_role': True, 'cr

In [ ]:
df = get_dataframe("AD_SALES_IMP")

In [ ]:
df_copy = df_all.copy()

In [ ]:
df_all.info()

In [ ]:
#df.columns = df.columns.str.lower()
#df_all.columns = df_all.columns.str.lower()
#df.info()

In [ ]:
df_all.columns

# Exploratory data analysis

In [ ]:
cat_col=['AD_DATE', 'SITE_ID','ADVERTISER_ID', 'ORDER_ID','AD_TYPE', 'AD_FORMAT', 'AD_MEDIA_TYPE',
         'DEVICE_TYPE', 'CITY', 'LINE_ITEM_GROUP', 'LINE_ITEM_TYPE', 'MONETIZATION_CHANNEL','OS_TYPE']
scat_col = ['SITE_ID','AD_TYPE', 'AD_FORMAT', 'DEVICE_TYPE', 'ADVERTISER_ID',
            'LINE_ITEM_GROUP', 'LINE_ITEM_TYPE', 'OS_TYPE','MONETIZATION_CHANNEL']

num_col=list(df_all.select_dtypes(np.number).columns)

In [ ]:
fig, ax = plt.subplots(4,2, figsize=(14,12))
axes_ = [axes_row for axes in ax for axes_row in axes]

for i,col in enumerate(scat_col):
    sns.countplot(data=df_all,x=col,ax=axes_[i])
    if col=='ADVERTISER_ID':
        plt.xticks(rotation=90)
    plt.tight_layout()

In [ ]:
for i in num_col:
    if i!='TOTAL_REVENUE':
        sns.scatterplot(data=df_all,x=i,y='TOTAL_REVENUE')
        plt.show()

In [ ]:
#df.max()

In [ ]:
for col in (scat_col):
    title='Relationship of '+col+' with total_revenue'
    plt.figure(figsize=(6,4))
    sns.barplot(y=df_all['TOTAL_REVENUE'],x=df_all[col])
    if col=='ADVERTISER_ID':
        plt.xticks(rotation=90)
    plt.title(title)
    plt.show()

In [ ]:
for col in (scat_col):
    title='Relationship of '+ col +' with total_impressions'
    plt.figure(figsize=(6,4))
    sns.barplot(y=df_all['TOTAL_IMPRESSIONS'],x=df_all[col],)
    if col=='ADVERTISER_ID':
        plt.xticks(rotation=90)
    plt.title(title)
    plt.show()

In [ ]:
for i in cat_col:
    df_all[i]=df_all[i].astype('object')

In [ ]:
df_all.drop(['AD_UNIT_ID','REVENUE_SHARE_PERCENT','AD_TYPE_ID','SITE_ID','ADVERTISER_ID',
        'AD_DATE','GEO_ID','ORDER_ID', 'AD_TYPE', 'AD_FORMAT', 'AD_MEDIA_TYPE', 'LINE_ITEM_GROUP',
            'CITY', 'CITY_CODE', 'DEVICE_CATEGORY_ID', 'LINE_ITEM_TYPE_ID', 'OS_ID',
       'MONETIZATION_CHANNEL_ID','POPULATION', 'CITY_LAT', 'CITY_LON'],axis=1,inplace=True)

In [ ]:
df_all.columns

In [ ]:
df_all.select_dtypes(object)

In [ ]:
for i in df_all.select_dtypes(object).columns:
    pd.crosstab(df_all['MONETIZATION_CHANNEL'],df_all[i]).plot(kind='bar')
    plt.show()

# Predictive Modelling

In [ ]:
import joblib

In [ ]:
Xd = df_all.drop('TOTAL_REVENUE',axis=1)
y = df_all['TOTAL_REVENUE']

In [ ]:
Xd.info()

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(Xd,y,test_size=0.2,random_state=4)

In [ ]:
pc_col = ['TOTAL_IMPRESSIONS', 'VIEWABLE_IMPRESSIONS', 'MEASURABLE_IMPRESSIONS']

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('scaler', StandardScaler()),
            ('pca', PCA(n_components=2))
        ]), pc_col),
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['DEVICE_TYPE', 'LINE_ITEM_TYPE', 'OS_TYPE',
       'MONETIZATION_CHANNEL'])
    ])

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor())
])

In [ ]:
models = [

    {
        'name': 'RandomForestRegressor',
        'regressor': [RandomForestRegressor()],
        'regressor__n_estimators': [50],
        'regressor__max_depth': [10],
        'regressor__min_samples_split': [2],
        'regressor__min_samples_leaf': [1],
        'regressor__bootstrap': [True]
    },
]

In [ ]:
best_estimators = []
for model_params in models:
    model_name = model_params.pop('name')  # Extract the model name
    grid_search = GridSearchCV(pipeline, model_params, cv=3, scoring='r2', n_jobs=-1)
    grid_search.fit(X_train, y_train)
    best_estimator = grid_search.best_estimator_
    best_estimators.append(best_estimator)
    print(f"Training completed for model {model_name}")
    
    # Save the best model
    joblib.dump(best_estimator, f'best_model_{model_name}.pkl')
    print(f"Best model {model_name} saved to best_model_{model_name}.pkl")

In [ ]:
results = []
for estimator in best_estimators:
    y_pred_train = estimator.predict(X_train)
    y_pred_test = estimator.predict(X_test)
    mse = mean_squared_error(y_test, y_pred_test)
    r2 = r2_score(y_test, y_pred_test)
    results.append({
        'model': estimator.named_steps['regressor'].__class__.__name__,
        'best_params': estimator.named_steps['regressor'].get_params(),
        'mse': mse,
        'r2': r2
    })

results_df = pd.DataFrame(results)
results_df

In [ ]:
df_all['PREDICTED_REVENUE'] = best_estimator.predict(Xd)
df_copy['PREDICTED_REVENUE'] = best_estimator.predict(Xd)

In [ ]:
df_copy

In [ ]:
#check

print(df_copy['PREDICTED_REVENUE'].sum())
print(df_copy['TOTAL_REVENUE'].sum())

# Pushing Model output to Snowflake

In [ ]:
import os
from snowflake.snowpark.session import Session
user = os.getenv("user")
warehouse = os.getenv("warehouse")
schema= os.getenv("schema")
database = os.getenv("database")
role =  os.getenv("role")
account =  os.getenv("account")
password= os.getenv("password")

connection_params = dict(user=user, 
                         password=password, 
                         account=account, 
                         warehouse=warehouse, 
                         database=database,
                         schema=schema, 
                         role=role)

session = Session.builder.configs(connection_params).create()

session.sql('use warehouse {};'.format(warehouse)).collect()

session.sql('use database {};'.format(database)).collect()

session.sql('use schema {}.{};'.format(database, schema)).collect()

In [ ]:
df_snowflake = session.createDataFrame(df_copy.values.tolist(),
        schema = df_copy.columns.tolist())

df_snowflake.write.mode("overwrite").save_as_table("ME_DB.ME_AD_SALES_SCHEMA.AD_TECH_OUTPUT")

In [ ]:
df_copy

# Model Registrartion using fosforml SDK

In [ ]:
#Snowpark lib
from snowflake.snowpark import Session
from fosforio import snowflake
from sklearn.pipeline import Pipeline
from fosforml import *
from fosforml.constants import MLModelFlavours
import requests

In [ ]:
@scoring_func
def score(model, request):

    import json
    payload = request.json["payload"]
    if isinstance(request.json["payload"],str):
        payload_data = eval(payload)
        if isinstance(payload_data['TOTAL_IMPRESSIONS'], int):
                data_json = eval(payload)
                data = pd.DataFrame([data_json])
                prediction = pd.DataFrame(model.predict(data))
                return prediction[0].to_list()[0]
        elif isinstance(payload_data['TOTAL_IMPRESSIONS'], dict):
                data = pd.DataFrame(eval(payload))
                prediction = pd.DataFrame(model.predict(data))
                return prediction[0].tolist()
        elif isinstance(payload_data['TOTAL_IMPRESSIONS'], list):
                data = pd.DataFrame(payload_data)
                prediction = pd.DataFrame(model.predict(data))
                return prediction.tolist()
    return "This method is not allowed"

In [ ]:
import requests

payload = str(X_test.iloc[1:3].to_dict())
req = requests.Request()
req.json = {"payload": payload}
print(score(best_estimator, req))

In [ ]:
payload

In [ ]:
import requests

payload = str(X_test.iloc[1].to_dict())
req = requests.Request()
req.json = {"payload": payload}
print(score(best_estimator, req))

In [ ]:
payload

# Sample Payload

In [ ]:
## registering the model in Fosfor.
model_reg = register_model(best_estimator,
               score, 
               name="Ad_Sales_Prediction_Model", 
               description="Ad_Sales_Prediction_RandomForest_Model",
               flavour=MLModelFlavours.sklearn,
               model_type="regression",
               #init_script="\\n pip install fosforml \\n pip install fosforio[snowflake] \\n pip install sklearn\\n pip install snowflake-connector-python[pandas]",
               init_script="\\n pip install fosforml \\n pip install fosforio[snowflake] \\n pip install seaborn \\n pip install snowflake-connector-python[pandas] \\n pip install joblib==1.3.2 scikit-learn=1.3.2",
               y_true=y_test,
               y_pred=y_pred_test,
               #prob=y_prob,
               features=X_train.columns,
               input_type="json", 
               explain_ai=True,
               x_train=X_train, 
               x_test=X_test, 
               y_train=y_train,
               y_test=y_test,
               feature_names=X_train.columns.tolist(),
               original_features=X_train.columns.tolist(),
               feature_ids=X_train.columns,
               kyd=True, kyd_score = True)

In [ ]:
model_reg